# Uso de QC_Financial (Parte 2)

Se importa `QC_Financial_3` que es la versión compilada para Python3.

In [1]:
from finrisk import QC_Financial_3 as Qcf

El siguiente diccionario se utiliza para dar formato a las columnas de los `pandas.DataFrames` que se utilizarán.

In [2]:
format_dict = {'nominal': '{0:,.2f}', 'amort': '{0:,.2f}', 'interes': '{0:,.2f}', 'flujo': '{0:,.2f}',
               'icp_inicial': '{0:,.2f}', 'icp_final': '{0:,.2f}',
               'valor_tasa': '{0:,.4%}', 'spread': '{0:,.4%}', 'gearing': '{0:,.2f}',
               'amort_moneda_pago': '{0:,.2f}', 'interes_moneda_pago': '{0:,.2f}', 'valor_indice_fx': '{0:,.2f}'}

## Legs
Los objetos de tipo `Leg` son una lista (o vector) de objetos `Cashflow` y representan una pata de un instrumento financiero. un objeto de tipo `Leg` puede construirse *a mano* es decir, dando de alta cashflows y agregándolos uno a uno o con algunos métodos de conveniencia cuyo funcionamiento se mostrará.
### Construcción Manual
Se verá como construir un `Leg` con 2 `SimpleCashflow` de forma *manual*. En particular, este objeto `Leg` podría representar una operación FX por entrega física.

In [3]:
leg = Qcf.Leg()
fecha_vcto = Qcf.QCDate(20, 9, 2018)
simple_cashflow_1 = Qcf.SimpleCashflow(fecha_vcto,  # fecha del flujo
                                       100,         # monto
                                       Qcf.QCCLP()) # moneda

simple_cashflow_2 = Qcf.SimpleCashflow(fecha_vcto,  # fecha del flujo
                                       -70000,      # monto
                                       Qcf.QCUSD()) # moneda
leg.append_cashflow(simple_cashflow_1)
leg.append_cashflow(simple_cashflow_2)

In [4]:
# Se observa el resultado
num = leg.size()
for i in range(0, num):
    print(Qcf.show(leg.get_cashflow_at(i)))

('2018-09-20', 100.0, 'CLP')
('2018-09-20', -70000.0, 'USD')


### Construcción Asistida de un `FixedRateLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [5]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(5, 11, 2019)
fecha_final = Qcf.QCDate(31, 5, 2023)
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(Qcf.QCDate(31, 12, 2019))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = Qcf.QCInterestRate(.03, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLF()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

In [6]:
dl = Qcf.DateList()

In [7]:
Qcf.first(fecha_inicio.month_diff_day_remainder(fecha_final, dl, bus_adj_rule))

42

In [8]:
Qcf.second(fecha_inicio.month_diff_day_remainder(fecha_final, dl, bus_adj_rule))

26

Se puede lograr una visualización mucho mejor del resultado utilizando un Dataframe de pandas y el método `show`.

In [9]:
# Se importa pandas de la forma usual
import pandas as pd

# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(Qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2019-12-30,2019-12-30,"100,000.00",0.00,458.33,False,458.33,CLF,3.0000%,LinAct360
1,2019-12-30,2020-06-30,2020-06-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
2,2020-06-30,2020-12-31,2020-12-31,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
3,2020-12-31,2021-06-30,2021-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
4,2021-06-30,2021-12-31,2021-12-31,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
5,2021-12-31,2022-06-30,2022-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
6,2022-06-30,2022-12-30,2022-12-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
7,2022-12-30,2023-06-30,2023-06-30,"100,000.00","100,000.00","1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360


In [10]:
t = 1 # i = 264 en la simulación t = 264 / 264
T = []
for i in range(fixed_rate_leg.size()):
    cshflw = fixed_rate_leg.get_cashflow_at(i)
    fecha = cshflw.get_settlement_date()
    print(fecha)
    T.append(fecha_inicio.day_diff(fecha) / 365.0)
    print(cshflw.amount())

30-12-2019
458.3333333333384
30-6-2020
1524.9999999999986
31-12-2020
1533.3333333333421
30-6-2021
1508.3333333333337
31-12-2021
1533.3333333333421
30-6-2022
1508.3333333333337
30-12-2022
1524.9999999999986
30-6-2023
1516.6666666666772


### Construcción Asistida de un `FixedRateLeg2`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow2`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow2` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [11]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(4, 4, 2018)
fecha_final = Qcf.QCDate(1, 3, 2021) 
bus_adj_rule = Qcf.BusyAdjRules.NO
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(Qcf.QCDate(31, 12, 2018))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = Qcf.QCInterestRate(.03, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLF()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg_2 = Qcf.LegFactory.build_bullet_fixed_rate_leg_2(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

In [12]:
tabla = []
for i in range(fixed_rate_leg_2.size()):
    tabla.append(Qcf.show(fixed_rate_leg_2.get_cashflow_at(i)))
    
df2 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df2.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2018-04-04,2018-09-01,2018-09-03,"100,000.00",0.00,"1,250.00",False,"1,250.00",CLF,3.0000%,LinAct360
1,2018-09-01,2019-03-01,2019-03-01,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
2,2019-03-01,2019-09-01,2019-09-02,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
3,2019-09-01,2020-03-01,2020-03-02,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
4,2020-03-01,2020-09-01,2020-09-01,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
5,2020-09-01,2021-03-01,2021-03-01,"100,000.00","100,000.00","1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360


In [13]:
df2.dtypes

fecha_ini          object
fecha_fin          object
fecha_pago         object
nominal           float64
amort             float64
interes           float64
amort_es_flujo       bool
flujo             float64
moneda             object
valor_tasa        float64
tipo_tasa          object
dtype: object

### Construcción Asistida de un `CustomAmortFixedRateLeg`
En este ejemplo se construye un `Leg` donde la estructura de amortizaciones es customizada.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `CustomNotionalAndAmort`: vector de capital vigente y amortizaciones customizado
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [14]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(31, 1, 1969)
fecha_final = Qcf.QCDate(31, 1, 1974) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
custom_notional_amort = Qcf.CustomNotionalAmort()
custom_notional_amort.set_size(10)
for i in range(0, 10):
    custom_notional_amort.set_notional_amort_at(i, 1000.0 - i * 100.0, 100.0)
amort_es_flujo = False
tasa_cupon = Qcf.QCInterestRate(.03, Qcf.QC30360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLF()
es_bono = False

In [15]:
# Se da de alta el objeto
fixed_rate_custom_leg = Qcf.LegFactory.build_custom_amort_fixed_rate_leg(rp,
                                                                         fecha_inicio,
                                                                         fecha_final,
                                                                         bus_adj_rule,
                                                                         periodicidad,
                                                                         periodo_irregular,
                                                                         calendario,
                                                                         lag_pago,
                                                                         custom_notional_amort,
                                                                         amort_es_flujo,
                                                                         tasa_cupon,
                                                                         moneda)

In [16]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_custom_leg.size()):
    tabla.append(Qcf.show(fixed_rate_custom_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df3 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df3.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,1969-01-31,1969-07-31,1969-07-31,"1,000.00",100.00,15.00,False,15.00,CLF,3.0000%,Lin30360
1,1969-07-31,1970-01-30,1970-01-30,900.00,100.00,13.50,False,13.50,CLF,3.0000%,Lin30360
2,1970-01-30,1970-07-31,1970-07-31,800.00,100.00,12.00,False,12.00,CLF,3.0000%,Lin30360
3,1970-07-31,1971-01-29,1971-01-29,700.00,100.00,10.44,False,10.44,CLF,3.0000%,Lin30360
4,1971-01-29,1971-07-30,1971-07-30,600.00,100.00,9.05,False,9.05,CLF,3.0000%,Lin30360
5,1971-07-30,1972-01-31,1972-01-31,500.00,100.00,7.50,False,7.50,CLF,3.0000%,Lin30360
6,1972-01-31,1972-07-31,1972-07-31,400.00,100.00,6.00,False,6.00,CLF,3.0000%,Lin30360
7,1972-07-31,1973-01-31,1973-01-31,300.00,100.00,4.50,False,4.50,CLF,3.0000%,Lin30360
8,1973-01-31,1973-07-31,1973-07-31,200.00,100.00,3.00,False,3.00,CLF,3.0000%,Lin30360
9,1973-07-31,1974-01-31,1974-01-31,100.00,100.00,1.50,False,1.50,CLF,3.0000%,Lin30360


### Construcción Asistida de un `FixedRateMultiCurrencyLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateMultiCurrencyCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateMultiCurrencyCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago.
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [17]:
# Primero se debe dar de alta un FXRateIndex
usd = Qcf.QCUSD()
clp = Qcf.QCCLP()
usdclp = Qcf.FXRate(usd, clp)
one_d = Qcf.Tenor('1D')
usdclp_obs = Qcf.FXRateIndex(usdclp, 'USDOBS', one_d, one_d, calendario)

# Luego se dan de alta los otros parámetros requeridos para la construcción
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(31, 1, 1969)
fecha_final = Qcf.QCDate(31, 1, 1974) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.NO
lag_pago = 0
es_bono = False

# Se da de alta el objeto
fixed_rate_mccy_leg = Qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(rp,
                                                                      fecha_inicio,
                                                                      fecha_final,
                                                                      bus_adj_rule,
                                                                      periodicidad,
                                                                      periodo_irregular,
                                                                      calendario,
                                                                      lag_pago,
                                                                      nominal,
                                                                      amort_es_flujo,
                                                                      tasa_cupon,
                                                                      usd,
                                                                      clp,
                                                                      usdclp_obs,
                                                                      0,
                                                                      es_bono)

In [18]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_mccy_leg.size()):
    tabla.append(Qcf.show(fixed_rate_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda_nominal',
            'valor_tasa', 'tipo_tasa', 'fecha_fijacion_fx', 'moneda_pago', 'indice_fx', 'valor_indice_fx', 'amort_moneda_pago',
           'interes_moneda_pago']
df4 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df4.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-07-31,1969-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1969-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
1,1969-07-31,1970-01-30,1970-01-30,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1970-01-30,CLP,USDOBS,1.00,0.00,"1,500.00"
2,1970-01-30,1970-07-31,1970-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1970-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
3,1970-07-31,1971-01-29,1971-01-29,"100,000.00",0.00,"1,491.67",False,"1,491.67",USD,3.0000%,Lin30360,1971-01-29,CLP,USDOBS,1.00,0.00,"1,491.67"
4,1971-01-29,1971-07-30,1971-07-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",USD,3.0000%,Lin30360,1971-07-30,CLP,USDOBS,1.00,0.00,"1,508.33"
5,1971-07-30,1972-01-31,1972-01-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1972-01-31,CLP,USDOBS,1.00,0.00,"1,500.00"
6,1972-01-31,1972-07-31,1972-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1972-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
7,1972-07-31,1973-01-31,1973-01-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1973-01-31,CLP,USDOBS,1.00,0.00,"1,500.00"
8,1973-01-31,1973-07-31,1973-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1973-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
9,1973-07-31,1974-01-31,1974-01-31,"100,000.00","100,000.00","1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1974-01-31,CLP,USDOBS,1.00,"100,000.00","1,500.00"


### Construcción Asistida de un `BulletIborLeg`
En este ejemplo se construye un `Leg` con `IborCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal y de los flujos
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IborCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_ibor_leg(...)`.

In [19]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(31, 1, 1969)
fecha_final = Qcf.QCDate(31, 1, 1971) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('3M')
periodo_irregular_pago = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = Qcf.Tenor('3M')
periodo_irregular_fijacion = Qcf.StubPeriod.NO

# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('2d')
tenor = Qcf.Tenor('3m')
fixing_calendar = calendario
settlement_calendar = calendario
usd = Qcf.QCUSD()
libor_usd_3m = Qcf.InterestRateIndex(codigo,
                                    lin_act360,
                                    fixing_lag,
                                    tenor,
                                    fixing_calendar,
                                    settlement_calendar,
                                    usd)
# Fin índice
nominal = 1000000.0
amort_es_flujo = True 
moneda = usd
spread = .01
gearing = 1.0

ibor_leg = Qcf.LegFactory.build_bullet_ibor2_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     periodicidad_fijacion, periodo_irregular_fijacion,
                                                     calendario, lag_de_fijacion, libor_usd_3m,
                                                     nominal, amort_es_flujo, moneda, spread, gearing)

In [20]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(Qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df5 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df5.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-01-29,1969-04-30,"1,000,000.00",0.00,"2,472.22",True,"2,472.22",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-04-28,1969-07-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-07-29,1969-10-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1969-10-29,1970-01-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-01-28,1970-04-30,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-04-28,1970-07-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-07-29,1970-10-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1970-10-28,1971-01-29,"1,000,000.00","1,000,000.00","2,527.78",True,"1,002,527.78",USD,LIBORUSD3M,0.0000%,1.0000%,1.00,LinAct360


### Construcción Asistida de un `BulletIborMultiCurrencyLeg`
En este ejemplo se construye un `Leg` con `IborMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborMultiCurrencyCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal
- `float`: spread aditivo
- `gearing`: spread multiplicativo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

In [21]:
ibor_mccy_leg = Qcf.LegFactory.build_bullet_ibor_mccy_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                          periodo_irregular_pago, calendario, lag_pago,
                                                          periodicidad_fijacion, periodo_irregular_fijacion,
                                                          calendario, lag_de_fijacion, libor_usd_3m,
                                                          nominal, amort_es_flujo, usd, spread, gearing,
                                                          clp, usdclp_obs, 0)

In [22]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_mccy_leg.size()):
    ibor_mccy_leg.get_cashflow_at(i).set_fx_rate_index_value(10.0)
    tabla.append(Qcf.show(ibor_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago',
            'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda_nominal', 'codigo_indice_tasa', 'spread', 'gearing', 'valor_tasa', 'tipo_tasa',
            'fecha_fijacion_fx', 'moneda_pago', 'codigo_indice_fx', 'valor_indice_fx',
            'amort_moneda_pago', 'interes_moneda_pago']
df6 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df6.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,codigo_indice_tasa,spread,gearing,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-04-30,1969-01-29,1969-04-30,"1,000,000.00",0.00,"2,472.22",True,"24,722.22",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-04-30,CLP,USDOBS,10.00,0.00,"24,722.22"
1,1969-04-30,1969-07-31,1969-04-28,1969-07-31,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-07-31,CLP,USDOBS,10.00,0.00,"25,555.56"
2,1969-07-31,1969-10-31,1969-07-29,1969-10-31,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-10-31,CLP,USDOBS,10.00,0.00,"25,555.56"
3,1969-10-31,1970-01-30,1969-10-29,1970-01-30,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-01-30,CLP,USDOBS,10.00,0.00,"25,277.78"
4,1970-01-30,1970-04-30,1970-01-28,1970-04-30,"1,000,000.00",0.00,"2,500.00",True,"25,000.00",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-04-30,CLP,USDOBS,10.00,0.00,"25,000.00"
5,1970-04-30,1970-07-31,1970-04-28,1970-07-31,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-07-31,CLP,USDOBS,10.00,0.00,"25,555.56"
6,1970-07-31,1970-10-30,1970-07-29,1970-10-30,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-10-30,CLP,USDOBS,10.00,0.00,"25,277.78"
7,1970-10-30,1971-01-29,1970-10-28,1971-01-29,"1,000,000.00","1,000,000.00","2,527.78",True,"10,025,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-01-29,CLP,USDOBS,10.00,"10,000,000.00","25,277.78"


### Construcción Asistida de un `IcpClpLeg`
En este ejemplo se construye un `Leg` con `IcpClpCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClpCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clp_leg(...)`.

In [23]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(31, 1, 1969)
fecha_final = Qcf.QCDate(31, 1, 1974) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('3M')
periodo_irregular_pago = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0

icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     nominal, amort_es_flujo, spread, gearing)

In [24]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp_leg.size()):
    tabla.append(Qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df7 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df7.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLP,"10,000.00","10,000.00",1.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",1.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",1.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",1.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLP,"10,000.00","10,000.00",1.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",1.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",1.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",1.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",1.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",1.0000%,"2,527.78",1.0000%,1.00,LinAct360


### Construcción Asistida de un `IcpClp2Leg`


En este ejemplo se construye un `Leg` con `IcpClpCashflow2` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClpCashflow2` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clp2_leg(...)`.

In [25]:
# Se da de alta los parámetros requeridos
icp_clp2_leg = Qcf.LegFactory.build_bullet_icp_clp2_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     nominal, amort_es_flujo, spread, gearing, True)

In [26]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(Qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLP,"10,000.00","10,000.00",0.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLP,"10,000.00","10,000.00",0.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360


Se puede customizar la amortización de la siguiente forma:

In [27]:
cna = Qcf.CustomNotionalAmort()
cna.set_size(19)

In [28]:
cna.set_notional_amort_at(0, 1000000, 0)
for i in range(1, icp_clp2_leg.size()):
    prev_amort = cna.get_amort_at(i - 1)
    prev_notional = cna.get_notional_at(i - 1)
    if i == 10 or i == icp_clp2_leg.size() - 1:
        cna.set_notional_amort_at(i, prev_notional - prev_amort, 500000)
    else:
        cna.set_notional_amort_at(i, prev_notional - prev_amort, 0)

for i in range(icp_clp2_leg.size()):
    cshflw = icp_clp2_leg.get_cashflow_at(i)
    cshflw.set_nominal(cna.get_notional_at(i))
    cshflw.set_amortization(cna.get_amort_at(i))

In [29]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(Qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLP,"10,000.00","10,000.00",0.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLP,"10,000.00","10,000.00",0.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360


### Construcción Asistida de un `IcpClfLeg`
En este ejemplo se construye un `Leg` con `IcpClfCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClfCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clf_leg(...)`.

In [30]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(31, 1, 1969)
fecha_final = Qcf.QCDate(31, 1, 1974) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('3M')
periodo_irregular_pago = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0

icp_clf_leg = Qcf.LegFactory.build_bullet_icp_clf_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     nominal, amort_es_flujo, spread, gearing)

In [31]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clf_leg.size()):
    tabla.append(Qcf.show(icp_clf_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'uf_inicial', 'uf_final',
            'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df8 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df8.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,uf_inicial,uf_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",27000.000000,27000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
